In [56]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns; sns.set()
import matplotlib.ticker as mtick
import copy
# from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline
from datetime import datetime

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import keras
from tensorflow.python.keras import layers as layer_module
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.engine import input_layer
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.engine import training_utils
from tensorflow.python.keras.saving.saved_model import model_serialization
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.util import nest
from tensorflow.python.util import tf_inspect
from tensorflow.python.util.tf_export import keras_export
from sklearn.model_selection import train_test_split

In [205]:
test_df = pd.read_csv(r'C:\Users\Pat\Documents\Thinkful\Files\NHL salary test.csv', encoding = 'ISO-8859-1')
train_df = pd.read_csv(r'C:\Users\Pat\Documents\Thinkful\Files\NHL salary train.csv', encoding = 'ISO-8859-1')
salaries_df = pd.read_csv(r'C:\Users\Pat\Documents\Thinkful\Files\NHL salary test_salaries.csv', encoding = 'ISO-8859-1')

# Statt of season is Oct 12, 2016
def elapsed_days(start, end=datetime(2016,10,12)):
	""" calcualte the number of days start and end dates"""
	x = (end - start)
	return x.days

# Get ages for season
train_df['AgeStart'] = train_df.apply(lambda x: 
    elapsed_days(datetime.strptime(x['Born'], '%y-%m-%d')) ,axis=1)

test_df['AgeStart'] = test_df.apply(lambda x: 
    elapsed_days(datetime.strptime(x['Born'], '%y-%m-%d')) ,axis=1)

train_df['AgeStart'] = train_df['AgeStart']/365.25
test_df['AgeStart'] = test_df['AgeStart']/365.25



# This player has a ton of missing info and only 1 game played
# test_df = test_df[test_df['Last Name'] != 'Renouf']
# train_df = test_df[test_df['Last Name'] != 'Renouf']

# Drop columns that are of no use or redundant
drop_cols = ['City', 'Pr/St', 'Cntry', 'Last Name', 'First Name', 'Team', 'Born']

train_df.drop(drop_cols, axis = 1, inplace = True)
test_df.drop(drop_cols, axis = 1, inplace = True)
# Two average shot distance measures. Not sure what the difference is
# but one is non-numeric and has missing values so I dropeed it
test_df.drop('sDist.1', axis = 1, inplace = True)
train_df.drop('sDist.1', axis = 1, inplace = True)


for i in test_df.columns.unique():
    print(i)
# print(test_df.columns.unique())
# test_df['age_season_start']
# test_df[['age_season_start', 'Born', 'Last Name', 'First Name']]

train_df = train_df.dropna()

# train_df = train_df.reset_index()
# test_df = train_df.reset_index()

Nat
Ht
Wt
DftYr
DftRd
Ovrl
Hand
Position
GP
G
A
A1
A2
PTS
+/-
E+/-
PIM
Shifts
TOI
TOIX
TOI/GP
TOI/GP.1
TOI%
IPP%
SH%
SV%
PDO
F/60
A/60
Pct%
Diff
Diff/60
iCF
iCF.1
iFF
iSF
iSF.1
iSF.2
ixG
iSCF
iRB
iRS
iDS
sDist
Pass
iHF
iHF.1
iHA
iHDf
iMiss
iGVA
iTKA
iBLK
iGVA.1
iTKA.1
iBLK.1
BLK%
iFOW
iFOL
iFOW.1
iFOL.1
FO%
%FOT
dzFOW
dzFOL
nzFOW
nzFOL
ozFOW
ozFOL
FOW.Up
FOL.Up
FOW.Down
FOL.Down
FOW.Close
FOL.Close
OTG
1G
GWG
ENG
PSG
PSA
G.Bkhd
G.Dflct
G.Slap
G.Snap
G.Tip
G.Wrap
G.Wrst
CBar 
Post
Over
Wide
S.Bkhd
S.Dflct
S.Slap
S.Snap
S.Tip
S.Wrap
S.Wrst
iPenT
iPenD
iPENT
iPEND
iPenDf
NPD
Min
Maj
Match
Misc
Game
CF
CA
FF
FA
SF
SA
xGF
xGA
SCF
SCA
GF
GA
RBF
RBA
RSF
RSA
DSF
DSA
FOW
FOL
HF
HA
GVA
TKA
PENT
PEND
OPS
DPS
PS
OTOI
Grit
DAP
Pace
GS
GS/G
AgeStart


In [206]:
train_df.columns = train_df.columns.str.replace('/', '')
train_df.columns = train_df.columns.str.replace('+', 'pls')
train_df.columns = train_df.columns.str.replace('-', 'min')
train_df.columns = train_df.columns.str.replace('%', 'perc')
train_df.columns = train_df.columns.str.replace('.', '')
train_df.columns = train_df.columns.str.replace(' ', '')

In [207]:
# Missing values
total_missing = train_df.isnull().sum().sort_values(ascending=False)
percent_missing = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total_missing, percent_missing], axis=1, keys=['Total', 'Percent'])
missing_data = pd.concat([total_missing, percent_missing], axis=1, keys=['Total', 'Percent'], sort=False)
missing_data.head(15)

# The top three columns are for undrafted players

,Total,Percent
AgeStart,0,0.0
iSF,0,0.0
iTKA,0,0.0
iGVA,0,0.0
iMiss,0,0.0
iHDf,0,0.0
iHA,0,0.0
iHF1,0,0.0
iHF,0,0.0
Pass,0,0.0


In [222]:
cat_feat = ['Nat', 'Hand', 'Position']
rem_feat = cat_feat
rem_feat.append('Salary')
# rem_feat
num_feat = list(train_df.drop(cat_feat, axis=1).columns)

In [223]:
train, test = train_test_split(train_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.25)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

300 train examples
100 validation examples
100 test examples


In [224]:
train

,Salary,Nat,Ht,Wt,DftYr,DftRd,Ovrl,Hand,Position,GP,...,OPS,DPS,PS,OTOI,Grit,DAP,Pace,GS,GSG,AgeStart
269,5800000,USA,77,225,2004.0,1.0,5.0,R,RW,82,...,6.5,1.7,8.2,3165.35,199,6.6,114.6,80.6,1.00,30.116359
601,575000,FIN,71,203,2011.0,2.0,52.0,L,RW,5,...,-0.2,0.0,-0.1,209.31,25,15.0,103.3,-0.6,-0.13,23.594798
158,575000,USA,70,184,2008.0,2.0,40.0,L,D,2,...,0.0,0.0,-0.1,80.15,4,0.0,93.5,-0.6,-0.28,26.403833
48,925000,RUS,73,201,2015.0,1.0,7.0,L,D,82,...,1.7,3.9,5.7,3080.32,311,8.9,107.3,35.3,0.43,19.745380
309,1000000,FIN,73,193,2004.0,1.0,19.0,L,LW/RW,69,...,0.7,1.0,1.8,2895.21,83,9.8,117.3,18.0,0.26,30.209446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,575000,USA,73,198,2007.0,5.0,135.0,L,C,6,...,-0.2,0.0,-0.2,245.70,6,0.0,105.0,0.4,0.06,28.049281
320,1650000,CAN,76,215,2009.0,2.0,54.0,L,D,27,...,-0.3,0.5,0.2,1036.23,60,5.8,106.0,1.4,0.05,25.431896
401,800000,CAN,77,236,2010.0,1.0,10.0,R,D,6,...,0.1,0.0,0.1,241.92,35,8.0,95.5,-0.3,-0.05,24.479124
76,667500,USA,72,193,2012.0,6.0,167.0,R,RW,3,...,-0.1,0.0,0.0,123.26,1,0.0,102.5,-0.4,-0.12,22.677618


In [225]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Salary')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [226]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [227]:
train_ds

<BatchDataset shapes: ({Nat: (None,), Ht: (None,), Wt: (None,), DftYr: (None,), DftRd: (None,), Ovrl: (None,), Hand: (None,), Position: (None,), GP: (None,), G: (None,), A: (None,), A1: (None,), A2: (None,), PTS: (None,), plsmin: (None,), Eplsmin: (None,), PIM: (None,), Shifts: (None,), TOI: (None,), TOIX: (None,), TOIGP: (None,), TOIGP1: (None,), TOIperc: (None,), IPPperc: (None,), SHperc: (None,), SVperc: (None,), PDO: (None,), F60: (None,), A60: (None,), Pctperc: (None,), Diff: (None,), Diff60: (None,), iCF: (None,), iCF1: (None,), iFF: (None,), iSF: (None,), iSF1: (None,), iSF2: (None,), ixG: (None,), iSCF: (None,), iRB: (None,), iRS: (None,), iDS: (None,), sDist: (None,), Pass: (None,), iHF: (None,), iHF1: (None,), iHA: (None,), iHDf: (None,), iMiss: (None,), iGVA: (None,), iTKA: (None,), iBLK: (None,), iGVA1: (None,), iTKA1: (None,), iBLK1: (None,), BLKperc: (None,), iFOW: (None,), iFOL: (None,), iFOW1: (None,), iFOL1: (None,), FOperc: (None,), percFOT: (None,), dzFOW: (None,), d

In [228]:
def get_scal(feature):
  def minmax(x):
    mini = train[feature].min()
    maxi = train[feature].max()
    return (x - mini)/(maxi-mini)
  return(minmax)

In [229]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of heights:', feature_batch['G'])
  print('A batch of targets:', label_batch )

Every feature: ['Nat', 'Ht', 'Wt', 'DftYr', 'DftRd', 'Ovrl', 'Hand', 'Position', 'GP', 'G', 'A', 'A1', 'A2', 'PTS', 'plsmin', 'Eplsmin', 'PIM', 'Shifts', 'TOI', 'TOIX', 'TOIGP', 'TOIGP1', 'TOIperc', 'IPPperc', 'SHperc', 'SVperc', 'PDO', 'F60', 'A60', 'Pctperc', 'Diff', 'Diff60', 'iCF', 'iCF1', 'iFF', 'iSF', 'iSF1', 'iSF2', 'ixG', 'iSCF', 'iRB', 'iRS', 'iDS', 'sDist', 'Pass', 'iHF', 'iHF1', 'iHA', 'iHDf', 'iMiss', 'iGVA', 'iTKA', 'iBLK', 'iGVA1', 'iTKA1', 'iBLK1', 'BLKperc', 'iFOW', 'iFOL', 'iFOW1', 'iFOL1', 'FOperc', 'percFOT', 'dzFOW', 'dzFOL', 'nzFOW', 'nzFOL', 'ozFOW', 'ozFOL', 'FOWUp', 'FOLUp', 'FOWDown', 'FOLDown', 'FOWClose', 'FOLClose', 'OTG', '1G', 'GWG', 'ENG', 'PSG', 'PSA', 'GBkhd', 'GDflct', 'GSlap', 'GSnap', 'GTip', 'GWrap', 'GWrst', 'CBar', 'Post', 'Over', 'Wide', 'SBkhd', 'SDflct', 'SSlap', 'SSnap', 'STip', 'SWrap', 'SWrst', 'iPenT', 'iPenD', 'iPENT', 'iPEND', 'iPenDf', 'NPD', 'Min', 'Maj', 'Match', 'Misc', 'Game', 'CF', 'CA', 'FF', 'FA', 'SF', 'SA', 'xGF', 'xGA', 'SCF', 

In [230]:
feature_columns = []
for header in num_feat:
  scal_input_fn = get_scal(header)
  feature_columns.append(feature_column.numeric_column(header, normalizer_fn=scal_input_fn))

In [231]:
for feature_name in cat_feat:
  vocabulary = train_df[feature_name].unique()
  cat_c = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
  one_hot = feature_column.indicator_column(cat_c)
  feature_columns.append(one_hot)

In [241]:
feature_columns[0]


NumericColumn(key='Ht', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x000002491CAF3E18>)

In [233]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  layers.Dropout(0.2),
  
  layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_fit = model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

ValueError: in converted code:
    relative to C:\Users\Pat\Anaconda3\lib\site-packages\tensorflow_core\python\feature_column:

    dense_features.py:133 call
        self._state_manager)
    feature_column_v2.py:4360 get_dense_tensor
        return transformation_cache.get(self, state_manager)
    feature_column_v2.py:2608 get
        transformed = column.transform_feature(self, state_manager)
    feature_column_v2.py:4299 transform_feature
        transformation_cache, state_manager)
    feature_column_v2.py:3774 get_sparse_tensors
        transformation_cache.get(self, state_manager), None)
    feature_column_v2.py:2608 get
        transformed = column.transform_feature(self, state_manager)
    feature_column_v2.py:3751 transform_feature
        transformation_cache.get(self.key, state_manager))
    feature_column_v2.py:2600 get
        raise ValueError('Feature {} is not in features dictionary.'.format(key))

    ValueError: Feature Salary is not in features dictionary.
